# Tutorial on how to open, visualize and extract some features from a .mhd Image

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install SimpleITK

     |████████████████████████████████| 47.4MB 85kB/s 


In [ ]:
pip install panda

  Created wheel for panda: filename=panda-0.3.1-cp37-none-any.whl size=7260 sha256=f24bb85d34b6ba2207b15e0784ec70592971ed7377395031fc0d49c5d087a21c
  Stored in directory: /root/.cache/pip/wheels/c6/c8/45/06ed898b0bb401c1ff207dbb05b1587ff28860a236d98b1996
Successfully built panda


This Tutorial will show how to:
    - Open and read a .mhd image
    - Visualize a .mhd image
    - Read a list of candidates from a .csv file
    - Transform from world coordinates to voxel coordinates
    - Extract some features / patches of candidates and visualize them
To be able to run this tutorial some python libraries / modules need to be installed:
    - Simple ITK: a library for handling and processing medical images
    - Numpy: a fundamental package for scientific computing with Python
    - PIL (Python Imaging Library): a library for adding image processing capabilities to your Python interpreter 
    - Matplotlib: a plotting library for the Python programming language

We start importing required modules / libraries  using the import command from python

In [ ]:
import SimpleITK as sitk
import numpy as np
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

We define now a function to:
    - Open the image 
    - Store it into a numpy array
    - Extract the following info: Pixel Spacing, Origin
This function takes as input the name of the image and returns:
    - The array corresponding to the image (numpyImage)
    - Origin (numpyOrigin)
    - PixelSpacing (numpySpacing)

In [ ]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
     
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
     
    return numpyImage, numpyOrigin, numpySpacing

To be able to open and read the list of candidates, we need to use the csv python module. 
We define now a function to:
    - Open a csv file
    - Read a csv file
    - Save each line of a csv file
This functions takes as input the name of the csv file and returns:
    - A list of each line of the csv

In [ ]:
def readCSV(filename):
    lines = []
    with open(filename, "rb") as f:
        csvreader = csv.reader(f)
        for line in csvreader:
            lines.append(line)
    return lines

Since the coordinates of the candidates are given in World Coordinates, we now need to transform from world coordinates to voxel coordinates. 
We define now a function to do that. Please note that the transformation below is only valid if there is no rotation component in the transformation matrix. For all CT images in our dataset, there is no rotation component so that this formula can be used. 
This function takes as inputs:
    - The world coordinates
    - The origin
    - The pixel Spacing
This function returns:
    - Voxel coordinates (voxelCoord)

In [ ]:
def worldToVoxelCoord(worldCoord, origin, spacing):
     
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

We want to extract now some features from the candidates. We define some normalized planes to extract views from the candidates

In [ ]:
def normalizePlanes(npzarray):
     
    maxHU = 400.
    minHU = -1000.
 
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray>1] = 1.
    npzarray[npzarray<0] = 0.
    return npzarray

After having defined these auxiliary functions, we can now define the main part of our script.
First we:
    - Specify the path where the image (img_path) is 
    - Specificy the path where the file with the list of candidates is (cand_path)

In [ ]:
img_path  = '/content/gdrive/MyDrive/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd'
cand_path = '/content/gdrive/MyDrive/luna/CSVFILES/candidates.csv'

Using the function defined in line 2 we can:
    - Load the image
    - Extract the Origin
    - Extract the Pixel Spacing 

In [ ]:
# load image
numpyImage, numpyOrigin, numpySpacing = load_itk_image(img_path)
print (numpyImage.shape)
print (numpyOrigin)
print (numpySpacing)

(121, 512, 512)
[-335.209991 -195.       -198.100006]
[2.5        0.76171899 0.76171899]


Using the function defined in line 3 we can:
    - Load the csv file
    - Get the candidates 
Using the function defined in line 4 we can: 
    - Transform from world to voxel coordinates

In [ ]:
# load candidates
cands = pd.read_csv(cand_path)
print (cands)



                                                seriesuid  ...  class
0       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
1       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
2       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
3       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
4       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
...                                                   ...  ...    ...
551060  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551061  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551062  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551063  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551064  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0

[551065 rows x 5 columns]


In [ ]:
print(cands[1:])
  

                                                seriesuid  ...  class
1       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
2       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
3       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
4       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
5       1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  ...      0
...                                                   ...  ...    ...
551060  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551061  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551062  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551063  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0
551064  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  ...      0

[551064 rows x 5 columns]


In [ ]:
# get candidates
for cand in cands[1:]:
    worldCoord = np.asarray([float(cand[3]),float(cand[2]),float(cand[1])])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    voxelWidth = 65

ValueError: ignored

In [ ]:
# get candidates
for cand in cands[1:]:
    worldCoord = np.asarray([float(cand[3]),float(cand[2]),float(cand[1])])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    voxelWidth = 65

ValueError: ignored

Using the function defined in line 5 we can:
    - Extract patch for each candidate in the list
    - Visualize each patch
    - Save each page as image in .tiff format

In [1]:
for cand in cands[1:]:
    worldCoord = np.asarray([float(cand[3]),float(cand[2]),float(cand[1])])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    voxelWidth = 65
    patch = numpyImage[voxelCoord[0],voxelCoord[1]-voxelWidth/2:voxelCoord[1]+voxelWidth/2,voxelCoord[2]-voxelWidth/2:voxelCoord[2]+voxelWidth/2]
    patch = normalizePlanes(patch)
    print ('data')
    print (worldCoord)
    print (voxelCoord)
    print (patch)
    outputDir = 'patches/'
    plt.imshow(patch, cmap='gray')
    plt.show()
    Image.fromarray(patch*255).convert('L').save(os.path.join(outputDir, 'patch_' + str(worldCoord[0]) + '_' + str(worldCoord[1]) + '_' + str(worldCoord[2]) + '.tiff'))
 

NameError: ignored